In [12]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from pickle import dump, load

# ==================================================
# 1. Carregar e preparar os dados
# ==================================================
print("\n" + "="*50)
print("CARREGAMENTO E PREPARAÇÃO DOS DADOS".center(50))
print("="*50)

dados = pd.read_excel('/content/drive/MyDrive/Aula/ProvaEscobar2/app_data.xlsx')

# Converter colunas numéricas (substituir vírgulas por pontos)
colunas_numericas = ['Age', 'BMI', 'Height', 'Weight', 'Appendix_Diameter', 'Body_Temperature',
                    'WBC_Count', 'Neutrophil_Percentage', 'RBC_Count', 'Hemoglobin',
                    'RDW', 'Thrombocyte_Count', 'CRP']
for col in colunas_numericas:
    dados[col] = pd.to_numeric(dados[col].astype(str).str.replace(',', '.'), errors='coerce')

# Remover tratamento não relevante
dados = dados[dados.Management != 'simultaneous appendectomy']

# Remover colunas desnecessárias
colunas_remover_fixas = [
    'Segmented_Neutrophils', 'US_Number', 'Appendix_Wall_Layers', 'Target_Sign',
    'Appendicolith', 'Perfusion', 'Perforation', 'Surrounding_Tissue_Reaction',
    'Appendicular_Abscess', 'Abscess_Location', 'Pathological_Lymph_Nodes',
    'Lymph_Nodes_Location', 'Bowel_Wall_Thickening', 'Conglomerate_of_Bowel_Loops',
    'Ileus', 'Coprostasis', 'Meteorism', 'Enteritis', 'Gynecological_Findings'
]

colunas_unnamed = [col for col in dados.columns if col.startswith('Unnamed:')]
colunas_remover = colunas_remover_fixas + colunas_unnamed
colunas_para_remover = [col for col in colunas_remover if col in dados.columns]

if colunas_para_remover:
    dados = dados.drop(columns=colunas_para_remover)
    print("\n➡ Colunas removidas do dataset:")
    print(", ".join(colunas_para_remover))
else:
    print("Nenhuma coluna para remover foi encontrada no DataFrame")

# ==================================================
# 2. Tratamento de valores faltantes
# ==================================================
print("\n" + "-"*50)
print("TRATAMENTO DE VALORES FALTANTES".center(50))
print("-"*50)

# Preencher colunas numéricas básicas
dados['Height'] = dados['Height'].fillna(dados['Height'].mean())
dados['Weight'] = dados['Weight'].fillna(dados['Weight'].mean())
dados['BMI'] = dados.apply(lambda row: row['Weight']/(row['Height']**2) if pd.isna(row['BMI']) else row['BMI'], axis=1)

# Preencher outras colunas numéricas com média
for col in ['Age', 'Appendix_Diameter', 'Body_Temperature', 'WBC_Count',
            'Neutrophil_Percentage', 'RBC_Count', 'Hemoglobin', 'RDW',
            'Thrombocyte_Count', 'CRP']:
    dados[col] = dados[col].fillna(dados[col].mean())

# Preencher colunas categóricas com moda
colunas_categoricas = ['Sex', 'Management', 'Severity', 'Diagnosis_Presumptive', 'Diagnosis',
                      'Appendix_on_US', 'Migratory_Pain', 'Lower_Right_Abd_Pain',
                      'Contralateral_Rebound_Tenderness', 'Coughing_Pain', 'Nausea',
                      'Loss_of_Appetite', 'Neutrophilia', 'Ketones_in_Urine', 'RBC_in_Urine',
                      'WBC_in_Urine', 'Dysuria', 'Stool', 'Peritonitis', 'Psoas_Sign',
                      'Ipsilateral_Rebound_Tenderness', 'US_Performed', 'Free_Fluids']
for col in colunas_categoricas:
    dados[col] = dados[col].fillna(dados[col].mode()[0])

print("✔ Valores faltantes tratados com sucesso")

# ==================================================
# 3. Normalização dos dados
# ==================================================
print("\n" + "-"*50)
print("NORMALIZAÇÃO DOS DADOS".center(50))
print("-"*50)

# Separar dados numéricos e categóricos
dados_num = dados.select_dtypes(exclude=['object'])
dados_cat = dados.select_dtypes(include=['object'])

# Normalizar dados numéricos
scaler = MinMaxScaler()
dados_num_normalizados = scaler.fit_transform(dados_num)
dados_num = pd.DataFrame(dados_num_normalizados, columns=dados_num.columns)

# Codificar dados categóricos (one-hot encoding)
dados_cat = pd.get_dummies(dados_cat, dtype='int')

# Juntar dados normalizados
dados = dados_num.join(dados_cat)
dados.fillna(0, inplace=True)

print("✔ Dados normalizados e codificados")

# ==================================================
# 4. Balanceamento de classes com SMOTE
# ==================================================
print("\n" + "-"*50)
print("BALANCEAMENTO DE CLASSES".center(50))
print("-"*50)

X = dados.drop(columns=['Management_conservative', 'Management_primary surgical',
                       'Management_secondary surgical', 'Severity_complicated',
                       'Severity_uncomplicated', 'Diagnosis_appendicitis',
                       'Diagnosis_no appendicitis'])
y_diagnosis = dados['Diagnosis_appendicitis']
y_severity = dados['Severity_complicated']
y_management = dados['Management_primary surgical']

smote = SMOTE(random_state=42, k_neighbors=1)
X_res_diag, y_res_diag = smote.fit_resample(X, y_diagnosis)
X_res_sev, y_res_sev = smote.fit_resample(X, y_severity)
X_res_mgt, y_res_mgt = smote.fit_resample(X, y_management)

print("✔ Dados balanceados com SMOTE")

# ==================================================
# 5. Modelagem e validação
# ==================================================
print("\n" + "="*50)
print("TREINAMENTO DOS MODELOS".center(50))
print("="*50)

rf = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')

# Dicionário para armazenar resultados
resultados = {}

for name, X_res, y_res in [('Diagnóstico', X_res_diag, y_res_diag),
                          ('Gravidade', X_res_sev, y_res_sev),
                          ('Tratamento', X_res_mgt, y_res_mgt)]:
    min_class_count = min(Counter(y_res).values())
    scores = cross_val_score(rf, X_res, y_res, cv=min_class_count, scoring='accuracy')
    resultados[name] = scores.mean()

    # Treinar e salvar modelo
    rf.fit(X_res, y_res)
    dump(rf, open(f'/content/drive/MyDrive/Aula/ProvaEscobar2/RandomForest_{name}.model', 'wb'))

    print(f"\n Modelo de {name}:")
    print(f"   - Acurácia média: {scores.mean():.2%}")
    print(f"   - Modelo salvo em: /content/drive/MyDrive/Aula/ProvaEscobar2/RandomForest_{name}.model")

# ==================================================
# 6. Previsão para novo paciente
# ==================================================
print("\n" + "="*50)
print("PREVISÃO PARA NOVO PACIENTE".center(50))
print("="*50)

novo_paciente = dados.tail(1).drop(columns=['Management_conservative', 'Management_primary surgical',
                                          'Management_secondary surgical', 'Severity_complicated',
                                          'Severity_uncomplicated', 'Diagnosis_appendicitis',
                                          'Diagnosis_no appendicitis'])

# Mapeamento para resultados mais descritivos
mapeamento_resultados = {
    'Diagnóstico': {1: 'Apêndicite confirmada', 0: 'Sem apêndicite'},
    'Gravidade': {1: 'Complicada', 0: 'Não complicada'},
    'Tratamento': {1: 'Cirúrgico', 0: 'Conservador'}
}

print("\n Resultados da previsão:")
for name in ['Diagnóstico', 'Gravidade', 'Tratamento']:
    modelo = load(open(f'/content/drive/MyDrive/Aula/ProvaEscobar2/RandomForest_{name}.model', 'rb'))
    predicao = modelo.predict(novo_paciente)
    resultado = mapeamento_resultados[name][predicao[0]]

    print(f"    {name}: {resultado} (confiança: {modelo.predict_proba(novo_paciente).max():.1%})")



       CARREGAMENTO E PREPARAÇÃO DOS DADOS        

➡ Colunas removidas do dataset:
Segmented_Neutrophils, US_Number, Appendix_Wall_Layers, Target_Sign, Appendicolith, Perfusion, Perforation, Surrounding_Tissue_Reaction, Appendicular_Abscess, Abscess_Location, Pathological_Lymph_Nodes, Lymph_Nodes_Location, Bowel_Wall_Thickening, Conglomerate_of_Bowel_Loops, Ileus, Coprostasis, Meteorism, Enteritis, Gynecological_Findings

--------------------------------------------------
         TRATAMENTO DE VALORES FALTANTES          
--------------------------------------------------
 Valores faltantes tratados com sucesso

--------------------------------------------------
              NORMALIZAÇÃO DOS DADOS              
--------------------------------------------------
 Dados normalizados e codificados

--------------------------------------------------
             BALANCEAMENTO DE CLASSES             
--------------------------------------------------
 Dados balanceados com SMOTE

       